# Importando as bibliotecas

In [ ]:
import pandas as pd
import sklearn

# Explorando a Base de Dados

In [ ]:
# Ler os dados e marcar as colunas com "?" como dados faltantes
adult = pd.read_csv("train_data.csv", na_values="?")

In [ ]:
# Ler os dados de teste
testAdult = pd.read_csv("test_data.csv", na_values="?")
nTestAdult = testAdult
nTestAdult.head()

,Id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,0,25,Private,120596,Bachelors,13,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,44,United-States
1,1,64,State-gov,152537,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States
2,2,31,Private,100135,Masters,14,Divorced,Exec-managerial,Not-in-family,White,Female,0,0,40,United-States
3,3,45,Private,189123,HS-grad,9,Never-married,Machine-op-inspct,Own-child,White,Male,0,0,40,United-States
4,4,64,Self-emp-inc,487751,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States


In [ ]:
adult.head()

,Id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,16280,34,Private,204991,Some-college,10,Divorced,Exec-managerial,Own-child,White,Male,0,0,44,United-States,<=50K
1,16281,58,Local-gov,310085,10th,6,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,<=50K
2,16282,25,Private,146117,Some-college,10,Never-married,Machine-op-inspct,Not-in-family,White,Male,0,0,42,United-States,<=50K
3,16283,24,Private,138938,Some-college,10,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,<=50K
4,16284,57,Self-emp-inc,258883,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,5178,0,60,Hungary,>50K


In [ ]:
adult.shape

(32560, 16)

In [ ]:
adult["income"].value_counts()

income
<=50K    24719
>50K      7841
Name: count, dtype: int64

Remover as colunas com dados faltantes

In [ ]:
nadult = adult.dropna()
# print(nadult.shape)

adult.head()

,Id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,16280,34,Private,204991,Some-college,10,Divorced,Exec-managerial,Own-child,White,Male,0,0,44,United-States,<=50K
1,16281,58,Local-gov,310085,10th,6,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,<=50K
2,16282,25,Private,146117,Some-college,10,Never-married,Machine-op-inspct,Not-in-family,White,Male,0,0,42,United-States,<=50K
3,16283,24,Private,138938,Some-college,10,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,<=50K
4,16284,57,Self-emp-inc,258883,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,5178,0,60,Hungary,>50K


## Primeiro Modelo

Vamos selecionar as variáveis que usaremos para o nosso classificador KNN. Para um primeiro teste, vamos usar apenas as variáveis numéricas.

In [ ]:
xAdult = nadult[["age", "education.num", "capital.gain", "capital.loss", "hours.per.week"]]

yAdult = list(nadult["income"])

Definindo o modelo do KNN. Inicialmente vamos usasr K=30. Mas testaremos outros valores em seguida.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=30)

from sklearn.model_selection import cross_val_score

Vamos usar a validação cruzada para validar o primeiro modelo

In [ ]:
scores = cross_val_score(knn, xAdult, yAdult, cv=10)

In [ ]:
avg = sum(scores)/10
print("Acurácia média da validação cruzada:", avg)

Acurácia média da validação cruzada: 0.8299459011665988


Usaremos esse valor de acurácia como uma base para tentarmos aprimorar o modelo.

### Normalizando os dados

Vamos tentar normalizar os valores para que valores de módulo muito grande não dominem os outros.

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.RobustScaler().fit(xAdult)

xAdultScaled = scaler.transform(xAdult)

Validando os dados com a nova escala temos:

In [ ]:
scores = cross_val_score(knn, xAdultScaled, yAdult, cv=10)

In [ ]:
avg = sum(scores)/10
print("Acurácia média da validação cruzada:", avg)

Acurácia média da validação cruzada: 0.8335598825928052


Nesse teste, ao que parece a acurácia aumenta bem pouco. Vamos **manter os dados sem normalizá-los**.

## Usando variáveis categóricas com One-Hot encoding

Vamos tentar agora icorporar classes categóricas usando one hot encoding

In [ ]:
nadult.head()

,Id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,16280,34,Private,204991,Some-college,10,Divorced,Exec-managerial,Own-child,White,Male,0,0,44,United-States,<=50K
1,16281,58,Local-gov,310085,10th,6,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,<=50K
2,16282,25,Private,146117,Some-college,10,Never-married,Machine-op-inspct,Not-in-family,White,Male,0,0,42,United-States,<=50K
3,16283,24,Private,138938,Some-college,10,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,<=50K
4,16284,57,Self-emp-inc,258883,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,5178,0,60,Hungary,>50K


Vamos codificar todas as colunas usando one-hot enconding, exceto a classe "native country"

In [ ]:
encodedAdult = pd.get_dummies(nadult, columns=["workclass", "education", "marital.status", "relationship", "occupation", "race", "sex"], dummy_na=True)

encodedAdultX = encodedAdult.drop(columns=["fnlwgt", "income", "Id", "native.country"])

Validando usando a validação cruzada obtemos:

In [ ]:
scores = cross_val_score(knn, encodedAdultX, yAdult, cv=10)
avg = sum(scores)/10
print("Acurácia média da validação cruzada:", avg)

Acurácia média da validação cruzada: 0.847982706748408


## Mantendo os dados faltantes

Nos exemplos acima retiramos os dados faltantes, vamos tentar considerá-los como uma categoria separada:

In [ ]:
encodedAdult = pd.get_dummies(adult, columns=["workclass", "education", "marital.status", "relationship", "occupation", "race", "sex"], dummy_na=True)

yAdult = adult["income"]

encodedAdultX = encodedAdult.drop(columns=["fnlwgt", "income", "Id", "native.country"])

Validando usando a validação cruzada obtemos:

In [ ]:
scores = cross_val_score(knn, encodedAdultX, yAdult, cv=10)
avg = sum(scores)/10
print("Acurácia média da validação cruzada:", avg)

Acurácia média da validação cruzada: 0.8524877149877149


A acurácia com os dados faltantes parece ter sido melhor! Vamos mantê-los, então.

## Vamos usar a validação cruzada para tentar escolher o melhor valor de k

In [ ]:
max_k = 1;
max_acc = 0;
for k in range(1,50):
    print("K:", k)
    knn = KNeighborsClassifier(n_neighbors=k)
    val_scores = cross_val_score(knn, encodedAdultX, yAdult, cv=10)
    avg_score = sum(val_scores)/10
    if avg_score > max_acc:
        max_acc = avg_score
        max_k = k
    print("avg_score:", avg_score)

print("O k com maior acurácia foi", max_k, "com acurácia de", max_acc)

# Fazendo predições:

Vamos usar os dados de treino para fazer predições sobre os dados de teste.

In [ ]:
# Vamos usar k=20 pois foi o que retornou melhor acurácia na validação cruzada.
knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(encodedAdultX,yAdult)

# Precisamos fazer o one-hot encoding dos dados de teste também para que eles fiquem compatíveis com os dados de treino
encodedTestAdult = pd.get_dummies(nTestAdult, columns=["workclass", "education", "marital.status", "relationship", "occupation", "race", "sex"], dummy_na = True)

encodedAdultXTest = encodedTestAdult.drop(columns=["fnlwgt", "Id", "native.country"])

YtestPred = knn.predict(encodedAdultXTest)


In [ ]:
resultado = pd.DataFrame({'Id':nTestAdult['Id'], 'income':YtestPred})

resultado.to_csv("pred.csv", index=False)